In [16]:
import torch.cuda

from src import  N_CLASSES, FEATURE_DIM

BATCH_SIZE = 32
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'device: {device}')

device: cpu


In [17]:
# 读数据
data = torch.load("data.pth")
feature = data['feature']
label = data['label']

In [18]:
# 生成训练集、测试集
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(feature, label, test_size=0.2, train_size=0.8, random_state=0)
x_train, x_test, y_train, y_test = x_train.to(device), x_test.to(device), y_train.to(device), y_test.to(device)

In [19]:
# 加载数据
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

test_dataset = TensorDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [20]:
# 定义网络
from model import Net
import torch.nn as nn

net = Net().to(device)

# 定义损失函数、优化器
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(),
                       lr=0.0001,
                       betas=(0.9, 0.999),
                       eps=1e-08,
                       weight_decay=0,
                       amsgrad=False)

In [21]:
# 训练网络
for epoch in range(1000):  # 数字越大，循环次数越多
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    if epoch % 10 == 0:
        print(f'epoch {epoch} loss: {running_loss / 1500:.10f}')
        running_loss = 0.0

print('Finished Training')

# 训练结束后保存网络参数
PATH = './model.pth'
torch.save(net.state_dict(), PATH)

epoch 0 loss: 0.7381000892
epoch 10 loss: 0.0784555322
epoch 20 loss: 0.0427763426
epoch 30 loss: 0.0298936598
epoch 40 loss: 0.0219812782
epoch 50 loss: 0.0185304847
epoch 60 loss: 0.0161231087
epoch 70 loss: 0.0135821077
epoch 80 loss: 0.0129703612
epoch 90 loss: 0.0111898305
epoch 100 loss: 0.0111980851
epoch 110 loss: 0.0093882697
epoch 120 loss: 0.0110214647
epoch 130 loss: 0.0090873142
epoch 140 loss: 0.0097399946
epoch 150 loss: 0.0090545689
epoch 160 loss: 0.0069340025
epoch 170 loss: 0.0091755603
epoch 180 loss: 0.0067685667
epoch 190 loss: 0.0068712495
epoch 200 loss: 0.0068794337
epoch 210 loss: 0.0074095086
epoch 220 loss: 0.0049696714
epoch 230 loss: 0.0068145098
epoch 240 loss: 0.0065242344
epoch 250 loss: 0.0058775752
epoch 260 loss: 0.0069919232
epoch 270 loss: 0.0063808572
epoch 280 loss: 0.0054384340
epoch 290 loss: 0.0054437259
epoch 300 loss: 0.0044803196
epoch 310 loss: 0.0046767764
epoch 320 loss: 0.0048795001
epoch 330 loss: 0.0052208376
epoch 340 loss: 0.0048750

In [22]:
# 测试模型
net = Net().to(device)
net.load_state_dict(torch.load(PATH))

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    # noinspection PyShadowingBuiltins
    for input, labels in test_loader:
        # calculate outputs by running images through the network
        outputs = net(input)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        # noinspection PyUnresolvedReferences
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {correct / total}')

Accuracy: 0.9734166666666667
